# An Introduction to Isolation Forests

This is an accompanying notebook for the package
[isotree](https://github.com/david-cortes/isotree)
presenting a short introduction to the Isolation Forest family of algorithms
as implemented in said package. For more information about it, see the GitHub page.
** *
## 1. Isolation Forests

## 1.1 Overview

Isolation Forest is an unsupervised decision-tree-based algorithm originally developed
for outlier detection in tabular data, which consists in splitting sub-samples of the data
according to some attribute/feature/column at random. The idea is that, the rarer the
observation, the more likely it is that a random split on some feature would put outliers alone
in one branch, and the fewer splits it will take to isolate (create a partition in which only
one point is present) an outlier observation like this.

The intuition behind it is very simple: if there is an outlier in the data and we pick a column
at random in which the value for the outlier point is different from the rest of the
observations, and then we select an arbitrary threshold uniformly at random within the range of
that column and divide all the points into two groups according to whether they are higher
or lower than the randomly-chosen threshold for that column, there is a higher chance that
the outlier point would end up in the smaller partition than in the larger partition.

Of course, outliers are typically not defined by just having one extreme value in one column,
and a good outlier detection method needs to look at the relationships between different
variables and their combinations. One potential way to do this is by building a so-called
"isolation tree", which consists of repeating the randomized splitting process described above
recursively (that is, we divide the points into two groups, then repeat the process in each
of the two groups that are obtained, and continue repeating it on the new groups until no
further split is possible or until meeting some other criteria).

Under this scheme, one can deduce that, the more common a point is, the more splits it will
take to leave the point alone or in a smaller group compared to uncommon points - as such,
one can think of the "isolation depth" (number of partitions that it takes to isolate a point,
hence the name of the algorithm) in the isolation trees as a metric by which to measure the
inlierness or outlierness of a point.

A single isolation tree has a lot of expected variability in the isolation depths that it is
expected to give to each observation, thus an ensemble of many such trees - an "isolation
forest" - may be used instead for better results, with the final score obtained by averaging
the results from many such trees.

There are many potential ways of improving upon the logic behind the procedure (for example,
extrapolating an isolation depth after reaching a certain limit) and the resulting score
can be standardized for easier usage, among many others - see the references for more details
about the methodology.

## 1.2 Why choose isolation forests over the alternatives

Compared to other outlier/anomaly detection methods such as "local outlier factor" or
"one-class support vector machines", isolation forests have advantages in that they are:

* Robust to the presence of outliers in training data.
* Robust to multi-modal distributions.
* Insensitive to the scales of variables.
* Much faster to fit.
* Invariant to the choice of distance metric (since it doesn't use a distance
metric in the first place).

Additionally, since they produce a standardized outlier metric for every point, such models
can be used for example to generate additional features for regression or classification models
or as a proxy for distribution density, for which not all outlier detection methods are
equally suitable (see the rest of the vignette for other potential uses of isolation forests).

## 1.3 An example in 1D

As a simple proof-of-concept test, one can think of producing random numbers from a normal
distribution and seeing what kinds of isolation depths would isolation trees assigning to
values from it:

In [17]:
%matplotlib inline
import numpy as np
import pandas as pd
import plotnine as p9



In [18]:
df=pd.read_excel(r'DiabetesDiagnosis.xls')

In [19]:
df

,Pregnancies,PG Concentration,Diastolic BP,Tri Fold Thick,Serum Ins,BMI,DP Function,Age,Diagnosis
0,6,148,72,35,0,33.6,0.627,50,0
1,1,85,66,29,0,26.6,0.351,31,1
2,8,183,64,0,0,23.3,0.672,32,0
3,1,89,66,23,94,28.1,0.167,21,1
4,0,137,40,35,168,43.1,2.288,33,0
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,1
764,2,122,70,27,0,36.8,0.340,27,1
765,5,121,72,23,112,26.2,0.245,30,1
766,1,126,60,0,0,30.1,0.349,47,0


In [20]:
from isotree import IsolationForest
model=IsolationForest()

In [21]:
model.fit(df[['Diagnosis']])

print(model.get_params())

{'sample_size': 'auto', 'ntrees': 500, 'ndim': 3, 'ntry': 1, 'categ_cols': None, 'max_depth': 'auto', 'ncols_per_tree': None, 'prob_pick_pooled_gain': 0.0, 'prob_pick_avg_gain': 0.0, 'prob_pick_full_gain': 0.0, 'prob_pick_dens': 0.0, 'prob_pick_col_by_range': 0.0, 'prob_pick_col_by_var': 0.0, 'prob_pick_col_by_kurt': 0.0, 'min_gain': 0.0, 'missing_action': 'auto', 'new_categ_action': 'auto', 'categ_split_type': 'auto', 'all_perm': False, 'coef_by_prop': False, 'recode_categ': False, 'weights_as_sample_prob': True, 'sample_with_replacement': False, 'penalize_range': False, 'standardize_data': True, 'scoring_metric': 'depth', 'fast_bratio': True, 'weigh_by_kurtosis': False, 'coefs': 'uniform', 'assume_full_distr': True, 'build_imputer': False, 'min_imp_obs': 3, 'depth_imp': 'higher', 'weigh_imp_rows': 'inverse', 'random_seed': 1, 'use_long_double': False, 'nthreads': -1, 'n_estimators': None, 'max_samples': None, 'n_jobs': None, 'random_state': None, 'bootstrap': None}


c:\Python310\lib\site-packages\isotree-0.5.17-py3.10-win-amd64.egg\isotree\__init__.py:1910: UserWarning: Model was meant to take 3 variables for each split, but data has 1 columns. Will decrease number of splitting variables to match number of columns.


In [22]:
df['scores'] = model.decision_function(df[['Diagnosis']])

df['anomaly_score'] = model.predict(df[['Diagnosis']])

df

,Pregnancies,PG Concentration,Diastolic BP,Tri Fold Thick,Serum Ins,BMI,DP Function,Age,Diagnosis,scores,anomaly_score
0,6,148,72,35,0,33.6,0.627,50,0,0.531690,0.531690
1,1,85,66,29,0,26.6,0.351,31,1,0.496051,0.496051
2,8,183,64,0,0,23.3,0.672,32,0,0.531690,0.531690
3,1,89,66,23,94,28.1,0.167,21,1,0.496051,0.496051
4,0,137,40,35,168,43.1,2.288,33,0,0.531690,0.531690
...,...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,1,0.496051,0.496051
764,2,122,70,27,0,36.8,0.340,27,1,0.496051,0.496051
765,5,121,72,23,112,26.2,0.245,30,1,0.496051,0.496051
766,1,126,60,0,0,30.1,0.349,47,0,0.531690,0.531690


In [23]:
df2=pd.read_csv(r'train.csv',index_col='PassengerId')

In [24]:
df2

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [25]:
model.fit(df2[['Survived']])

print(model.get_params())
df2['scores'] = model.decision_function(df2[['Survived']])

df2['anomaly_score'] = model.predict(df2[['Survived']])

df2


{'sample_size': 'auto', 'ntrees': 500, 'ndim': 3, 'ntry': 1, 'categ_cols': None, 'max_depth': 'auto', 'ncols_per_tree': None, 'prob_pick_pooled_gain': 0.0, 'prob_pick_avg_gain': 0.0, 'prob_pick_full_gain': 0.0, 'prob_pick_dens': 0.0, 'prob_pick_col_by_range': 0.0, 'prob_pick_col_by_var': 0.0, 'prob_pick_col_by_kurt': 0.0, 'min_gain': 0.0, 'missing_action': 'auto', 'new_categ_action': 'auto', 'categ_split_type': 'auto', 'all_perm': False, 'coef_by_prop': False, 'recode_categ': False, 'weights_as_sample_prob': True, 'sample_with_replacement': False, 'penalize_range': False, 'standardize_data': True, 'scoring_metric': 'depth', 'fast_bratio': True, 'weigh_by_kurtosis': False, 'coefs': 'uniform', 'assume_full_distr': True, 'build_imputer': False, 'min_imp_obs': 3, 'depth_imp': 'higher', 'weigh_imp_rows': 'inverse', 'random_seed': 1, 'use_long_double': False, 'nthreads': -1, 'n_estimators': None, 'max_samples': None, 'n_jobs': None, 'random_state': None, 'bootstrap': None}


c:\Python310\lib\site-packages\isotree-0.5.17-py3.10-win-amd64.egg\isotree\__init__.py:1910: UserWarning: Model was meant to take 3 variables for each split, but data has 1 columns. Will decrease number of splitting variables to match number of columns.


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,scores,anomaly_score
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.499125,0.499125
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.525471,0.525471
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.525471,0.525471
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.525471,0.525471
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.499125,0.499125
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0.499125,0.499125
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0.525471,0.525471
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0.499125,0.499125


In [14]:
df2

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,scores,anomaly_score
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.499125,0.499125
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.525471,0.525471
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.525471,0.525471
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.525471,0.525471
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.499125,0.499125
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0.499125,0.499125
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0.525471,0.525471
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0.499125,0.499125


In [26]:
df2.isnull().sum()

Survived           0
Pclass             0
Name               0
Sex                0
Age              177
SibSp              0
Parch              0
Ticket             0
Fare               0
Cabin            687
Embarked           2
scores             0
anomaly_score      0
dtype: int64